# Exercício 2 - Trabalho Prático 3

**Grupo 6:** 


Ruben Silva - pg57900

Luís Costa - pg55970

# Problema: 


1. Usando o protocolo  OT construído na questão anterior
    <!-- 1. Implemente o protocolo $\,\mathsf{sVOLE}\;$ (“subset vectorial oblivious linear evaluation”) -->
    2. Usando $\,\mathsf{sVOLE}\,$   implemente um protótipo de um protocolo  ZK-sVOLE usando equações polinomiais do 2º grau aleatoriamente geradas.


### imports

In [17]:
from sage.all import GF, vector, random_vector, ZZ, Integer
from random import randrange
import random
from sage import *


### Importar o Protocolo Anterior

In [18]:
def bernoulli_generator(epsilon, n=64):
    w = [random.randint(0, 1) for _ in range(n)]
    hat_w = sum(w[i] * (2**-(i+1)) for i in range(n))
    return 1 if hat_w <= epsilon else 0

def error(l, epsilon):
    F2 = GF(2)
    e = [F2(0)] * l
    for i in range(l):
        e[i] = bernoulli_generator(epsilon)
    return e

def gerar_sk(size):
    F2 = GF(2)
    return [F2(random.randint(0, 1)) for _ in range(size)]

def receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list):
    F2 = GF(2)
    s = [gerar_sk(lambda_) if k != b else None for k in range(N)]

    t = [[F2(0) for _ in range(N)] for _ in range(l)]

    for i in range(l):
        sum_t = F2(0)
        for k in range(N):
            if k != b:
                e_ik = F2(bernoulli_generator(epsilon))

                a_i = F2(0)
                for j in range(lambda_):
                    a_i += a_list[i][j] * s[k][j]
                
                t[i][k] = a_i + e_ik
                sum_t += t[i][k]

            else:
                t[i][k] = u_list[i] - sum(t[i][j] for j in range(N) if j != b)
        t[i][b] = u_list[i] - sum_t

    return s, t

def sender_transfer(m, t, a_list, l, N, delta):
    F2 = GF(2)
    # calcular o vetor de erro
    r = error(l, delta)

    # Calcular vetor a
    a = vector(F2, [0] * len(a_list[0]))
    for i in range(l):
        a+= r[i] * a_list[i]

    # Calcular vetor c
    c = []
    for k in range(N):
        c_k = m[k]
        for i in range(l):
            if r[i] == F2(1):
                c_k += t[i][k] * r[i]
        c.append(c_k)
    return a, c

def receiver_decipher(a, c, s, b, l, N, epsilon):
    F2 = GF(2)
    m_recovered = [None] * N
    for k in range(N):
        if k != b and s[k] is not None:
            a_sk = a * vector(F2, s[k])
            m_recovered[k] = c[k] + a_sk 
    return m_recovered

def recuperar_bits(s, b, a_list, t, m, bits_per_message, N, l, t_iterations, delta, epsilon):
    # Armazenar as estimativas dos bits recuperados
    results = [[[] for _ in range(bits_per_message)] for _ in range(N)]

    for bit_pos in range(bits_per_message):

        # Extrai o bit na posição bit_pos de cada mensagem 
        m_bit = [m[k][bit_pos] for k in range(N)]

        for _ in range(t_iterations):

            a, c = sender_transfer(m_bit, t, a_list, l, N, delta)
            m_rec = receiver_decipher(a, c, s, b, l, N, epsilon)

            #Itera sobre cada mensagem k para armazenar a estimativa do bit
            for k in range(N):
                if m_rec[k] is not None:
                    results[k][bit_pos].append(m_rec[k])
    return results  

def votacao(b,resultado, bits_per_message, N):
    F2 = GF(2)

    # Armazenar as estimativas dos bits recuperados
    m_final = [[None] * bits_per_message for _ in range(N)]
    for k in range(N):

        #conforme o protocolo, sb = NULL
        if k != b:
            for bit_pos in range(bits_per_message):
                # Conta o número de estimativas iguais a 1 para o bit
                ones = sum(1 for bit in resultado[k][bit_pos] if bit == 1)

                # Conta o número de estimativas iguais a 0 para o bit
                zeros = len(resultado[k][bit_pos]) - ones

                # Se o número de estimativas iguais a 1 for maior que o número de estimativas iguais a 0,
                # define o bit recuperado como 1, caso contrário, define como 0
                m_final[k][bit_pos] = F2(1) if ones > zeros else F2(0)
    return m_final

def verificar_igualdade(t, u_list):
    F2 = GF(2)
    for i in range(len(t)):
        t_sum = F2(sum(t[i][k] for k in range(len(t[i]))))
        if t_sum != F2(u_list[i]):
            print(f"Verification failed for i={i}: {t_sum} != {u_list[i]}")
            return False
    print("Verification successful.")
    return True



# Funções de conversão para LPN OT com F_p (adaptadas de ex1.ipynb)
def fp_to_bytes(x, p):
    # Converte elemento de F_p para bytes (LPN OT, usado em sVOLE)
    return bytes([int(x) % 256])  # Um byte para p ≤ 256

def bytes_to_fp(bytes_msg, p):
    # Converte bytes para elemento de F_p (LPN OT, usado em sVOLE)
    Fp = GF(p)
    return Fp(bytes_msg[0] % p)

def LPN(N, l, lambda_, epsilon, delta, b, t_iterations, m_bytes):
    F2 = GF(2)
    a_list = [random_vector(F2, lambda_) for _ in range(l)]
    u_list = [F2(random.randint(0, 1)) for _ in range(l)]
    
    bits_per_message = len(m_bytes[0]) * 8
    m_bits = [[F2((m_bytes[k][i // 8] >> (7 - (i % 8))) & 1) for i in range(bits_per_message)] for k in range(N)]
    
    s, t = receiver_choose(N, l, lambda_, epsilon, b, a_list, u_list)
    
    if not verificar_igualdade(t, u_list):
        return False, None
    
    resultado = recuperar_bits(s, b, a_list, t, m_bits, bits_per_message, N, l, t_iterations, delta, epsilon)
    
    m_final_bits = votacao(b, resultado, bits_per_message, N)
    
    m_final_bytes = [None] * N
    for k in range(N):
        if k != b and m_final_bits[k] is not None:
            bytes_rec = bytearray(len(m_bytes[0]))
            for i in range(bits_per_message):
                byte_idx = i // 8
                bit_idx = 7 - (i % 8)
                if m_final_bits[k][i] is not None:
                    bytes_rec[byte_idx] |= (int(m_final_bits[k][i]) << bit_idx)
            m_final_bytes[k] = bytes(bytes_rec)
    
    return True, m_final_bytes


### Ex 1

## Explicação do Vole

Para definir o protocolo vamos começar por fixar os parâmetros, $\,p\,$, $\,k\,$ e $\,\ell\,$, que determinam a estrutura algébrica onde o protocolo se desenvolve. O inteiro $\,p\,$ é um primo que determina o corpo $\,\mathbb{F}_p\,$, aqui designado simplesmente por $\,{F}\,$; o inteiro positivo $\,k\,$ é a dimensão de uma extensão de $\,F\,$ aqui representada por $\,E\,$; isto é, $\,E\simeq \mathbb{F}_{p^k}$ . 

Como sabemos, para uma uma variável genérica $\,\chi\,$, cada  $\,x\in E\,$  tem uma representação por um polinómio de coeficientes em $\,F\,$ de grau inferior a $\,k\,$ : isto é um elemento de $\,F[\chi]_k\,$.

Para além desses corpos vamos também considerar os espaços vectoriais $\,F^\ell\,$ e $\,E^\ell\,$. Os vetores em  $\,E^\ell\,$ podem ser vistos, também, como os coeficientes de polinómios numa variável $\,X\,$ e de grau menor do que $\,\ell\,$: isto é  elementos de $\,E[X]_\ell\,$.

Vamos também considerar codificações bijectivas  $\,\sigma\,$ de elementos de $\,F\,,\,E\,,\,E^\ell\,$ em strings de bits. Se $\,x\in F\,$  então $\,\sigma(x)\,$ é uma string de bits de comprimento igual ao tamanho de $\,p\,$ , representado por $\,|p|\,$.  Deste modo se $\,x\in E\,$ então temos $\,|\sigma(x)| = k\times|p|\,$ ; se $\,x\in E^\ell\,$ temos $\,|\sigma(x)| = \ell\times k \times |p|\,$.

Temos também a função de descodificação $\,\sigma^{-1}\,$ que recupera o objecto aritmético $\,x\,$ a partir da sua representação $\,\sigma(x)\,$ por uma string de bits.

### sVOLE

Para criar uma solução que responda  simultaneamente a todas estas questões, a partir de um factor de segurança $\,\lambda\,$ e vamos seguir os seguintes objectivos

1. Os “tags” $\,\mathsf{m}\,$ e $\,\mathsf{q}\,$ passam a ser vetores com $\,\ell\,$ componentes em $\,E\,$; como  elementos de $\,E^\ell\,$ o  o seu tamanho é substancialmente aumentado para  $\,|p|\times k\times \ell\,$;  se a dimensão $\,\ell\,$ for suficientemente grande, mesmo com $|p|$ e $\,k\,$ relativamente pequenos 
2. Os índices $\,z\,$ que identificam os elementos $\,t_z\,$ usados no protocolo OT   formam um subconjunto  $\,\mathcal{Z}\,\equiv\,\{z_1,z_2,\cdots\,z_N\}\,\subset E\,$ com $\,N  = \mathsf{poly}(\lambda)$  elementos.  
    O valor de $\,N\,$ e os elementos $\,\{z_n\}_{n\in[N]}\,$ são escolhidos de tal forma que, para todo $\,n\in [N]\,$ o conjunto $\,\mathcal{Z}\setminus\{z_n\}\,$ é uma base de $\,\mathbb{F}_{p^k}\,$ visto como espaço vectorial sobre $\,\mathbb{F}_p$.  
    Concretamente, cada $\,a\in E\,$ é uma combinação linear, com coeficientes em $\,F\,,$ dos elementos de $\,\mathcal{Z}\,$ e , cada $\,z_n\in\mathcal{Z}\,$ é uma combinação linear dos elementos de $\,\mathcal{Z}\setminus\{z_n\}\,$.
3. Os valores a autenticar $\,\overline{x}\,=\,\langle x_1, x_2,\cdots,x_\ell\rangle\,$ passam a ser também vectores de domensão $\,\ell\,$; isto é $\,\overline{x}\in F^{\ell}$.
4. A chave global $\,\Delta\,$ é escolhida, em cada instância do protocolo, dentro do  conjunto $\,\mathcal{Z}\,$ através da seleção de um índice $\,j\gets [N]\,$, e depois escolhendo $\,\Delta \equiv z_j\,$.
                                              $\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}$
5. Os valores $\,t_z\in F\,\,$ do protocolo OLE são substituídos por vectores; assim, para todo $\,i\in[\ell]\,$,  define-se um vector  $\,\overline{t}_i = \{t_{i,n}\}_{n\in[N]}\in\,F^\ell\,$ em que, 
    1. para $\,n>1\,$ , tem-se $\,t_{i,n}\,\gets\,F\,$ é gerado pseudo-aleatoriamente
    2. para  $\,n=1\,$  tem-se  $t_{i,1}\,\gets\, x_i - \sum_{n=2}^N\,t_{i,n}$.

Isto assegura que, para todo $\,i\in[\ell]\,$, se verifica $\,x_i \,=\,\sum_{n\in[N]}\,t_{i,n}$

### Dominio Algébrico

In [19]:
# Define parameters for sVOLE
p = 5  # Prime for the base field F = F_p (sVOLE)
k = 2  # Extension degree for E = F_{p^k} (sVOLE)
ell = 2  # Dimension of vectors (sVOLE)
N = 3  # Size of subset Z (sVOLE)

# Define parameters for LPN OT (from ex1.ipynb)
l = 12  # Number of oblivious criterion pairs (LPN OT)
lambda_ = 16  # Security parameter for secret keys (LPN OT)
epsilon = 0.001  # Error probability for Bernoulli generator (LPN OT)
delta = 1  # Parameter for error vector in sender_transfer (LPN OT)
t_iterations = 1000  # Number of iterations for OT robustness (LPN OT)

# Validate sVOLE parameters
if not ZZ(p).is_prime():
    raise ValueError(f"p = {p} must be a prime number")  # sVOLE validation
if k <= 0:
    raise ValueError(f"k = {k} must be a positive integer")  # sVOLE validation
order = p**k
if order < 2:
    raise ValueError(f"Order of the finite field p^k = {order} must be at least 2")  # sVOLE validation

# Define the base field F = F_p (sVOLE)
F = GF(p)
print(f"Base field F: {F}")

# Define the extension field E = F_{p^k} (sVOLE)
try:
    E = GF(p**k, name='z')  # Use 'z' consistently
    print(f"Extension field E: {E}")
except ValueError as e:
    print(f"Error creating finite field: {e}")
    raise

# Generate the subset Z ⊂ E (sVOLE)
alpha = E.gen()  # Generator of E (sVOLE)
Z = [E(1), alpha, alpha + 1]  # Z = {1, α, α + 1} (sVOLE)
print(f"Subconjunto Z: {Z}")
# Note: In a rigorous sVOLE implementation, verify that Z \ {z_n} is a basis of E over F

Base field F: Finite Field of size 5
Extension field E: Finite Field in z of size 5^2
Subconjunto Z: [1, z, z + 1]


### Sigma

In [20]:
def sigma(x):
    if x in F:
        return Integer(x)  # Representação como inteiro
    elif x in E:
        return x.vector()  # Representação como vetor em F^k
    elif x in E^ell:
        return [xi.vector() for xi in x]  # Vetor em E^ell como lista de vetores
    else:
        raise ValueError("Elemento inválido para codificação")

def sigma_inv(x, domain):
    if domain == F:
        return F(x)
    elif domain == E:
        return E(x)
    elif domain == 'E^ell':
        return vector(E, [E(xi) for xi in x])
    else:
        raise ValueError("Domínio inválido para decodificação")

`Prover`

Este agente conhece um vector $\,\overline{x}\in F^\ell\,$ , que é sua informação privada, e procede do seguinte modo 

1. para todo $\,i\in [\ell]$
    1. Gera o vector $\,\overline{t}_i\,$  como está indicado no ponto 5. acima,
    2. Disponibiliza $\,N\,$ mensagens $\,\{m_{i,n} \gets \sigma(t_{i,n})\}_{n\in [N]}\,$   para transferência num protocolo “oblivious transfer”  $\,{N\choose{N-1}}$ .
    3. Calcula o “tag”  $\;\mathsf{m}_i\,\gets\,\sum_{n\in[N]}\,(-z_n)\cdot t_{i,n}\,$.
2. Agrega num vector $\;\overline{\mathsf{m}}\in E^\ell\;$ os vários “tags”  $\,\{\mathsf{m}_i\}_{i\in [\ell]}$

In [21]:
def prover(x_bar, Z):
    ell = len(x_bar)
    N = len(Z)
    t_vectors = []
    messages = []
    m_tags = []

    for i in range(ell):
        # Gerar t_{i,n} para n = 2, ..., N
        #n > 1
        t_i = [F.random_element() for _ in range(1, N)]
        #n = 1
        t_i_1 = x_bar[i] - sum(t_i)
        t_i = [t_i_1] + t_i #concatenar t_i_1 no início da lista

        # Adicionar t_i à lista de vetores
        t_vectors.append(t_i)
        
        # Mensagens para OT
        m_i = [sigma(t_i[n]) for n in range(N)]
        messages.append(m_i)
        
        # Calcular tag m_i = sum_{n=1}^N (-z_n) * t_{i,n}
        m_i_tag = sum(-Z[n] * E(t_i[n]) for n in range(N))
        m_tags.append(m_i_tag)
    
    # Agregar mensagens m_bar \in E^ell
    m_bar = vector(E, m_tags)
    return t_vectors, messages, m_bar


`Verifier`

* $\forall$ 
  1. Gera $\,\Delta\,\gets\,\{\,\vartheta\,j\gets [N]\,\centerdot\, z_j\,\}\;$ que passa a ser designada por chave global. 
  2. para todo $\,i\in [\ell]$
      1. No $i$-ésimo  protocolo $\,{N}\choose{N-1}\,$-OT  inicializado pelo Prover, transfere as $\,N-1\,$ mensagens $\{m_{i,n}\}_{n\in [N]\setminus\{j\}}$ 
      2. Calcula  $\quad\mathsf{q}_i\,\gets\,\sum_{n\in [N]\setminus{\{j\}}}\,(\Delta-z_n)\cdot\sigma^{-1}(m_{i,n})$
  3. Agrega num único vector $\,\overline{\mathsf{q}}\in E^\ell\,$  todas as “tags” $\,\{\mathsf{q}_i\}_{i\in[\ell]}$
É trivial verificar que ambos os agentes agem honestamente se e só se  é válida a relação

$\,{\mathsf{q}}_i\,=\,{x}_i \cdot \Delta \,+\,{\mathsf{m}}_i$

para todo $\,i\in [\ell]\,$.

In [22]:
def verifier(messages, Z, newDelta=None):
    N = len(Z)
    ell = len(messages)
    
    # Escolher Delta = z_j aleatoriamente
    j = randrange(N)
    Delta = Z[j]

    if newDelta is not None:
        Delta = newDelta
        j = Z.index(Delta)
    
    # Simular OT: obter {m_{i,n}}_{n ≠ j}
    q_tags = []
    print("==============")
    for i in range(ell):
        print("Protocolo LPN OT - " + str(i))
        # Converter mensagens para bytes para o OT (LPN OT integration)
        m_i_bytes = [fp_to_bytes(messages[i][n], p) for n in range(N)]
        
        # Executar o OT para receber N-1 mensagens (LPN OT)
        correct, m_final = LPN(N, l, lambda_, epsilon, delta, j, t_iterations, m_i_bytes)
        if not correct:
            raise ValueError(f"OT falhou para i = {i}")
        
        # Decodificar mensagens recebidas (LPN OT to sVOLE)
        m_i_subset = []
        z_subset = []
        for n in range(N):
            if n != j and m_final[n] is not None:
                m_i_subset.append(bytes_to_fp(m_final[n], p))
                z_subset.append(Z[n])
        
        print("==============")
        # Calcular q_i = sum_{n ≠ j} (Delta - z_n) * sigma^{-1}(m_{i,n})
        q_i = sum((Delta - z_subset[n]) * E(sigma_inv(m_i_subset[n], F)) for n in range(len(z_subset)))
        q_tags.append(q_i)
    
    q_bar = vector(E, q_tags)
    return Delta, q_bar, j


In [23]:
def verificar(x_bar, Delta, q_bar, m_bar):
    for i in range(ell):
        expected = E(x_bar[i]) * Delta + m_bar[i]
        if q_bar[i] != expected:
            print(f"Verificação falhou para i={i}: {q_bar[i]} != {expected}")
            return False
    print("Verificação bem-sucedida.")
    return True

In [24]:
#def verifier_ideal(messages, Z, newDelta=None):
#    N = len(Z)
#    ell = len(messages)
#    j = randrange(N)
#    Delta = Z[j]
#    if newDelta is not None:
#        Delta = newDelta
#        j = Z.index(Delta)
#    
#    q_tags = []
#    for i in range(ell):
#        m_i_subset = [messages[i][n] for n in range(N) if n != j]
#        z_subset = [Z[n] for n in range(N) if n != j]
#        q_i = sum((Delta - z_subset[n]) * E(sigma_inv(m_i_subset[n], F)) for n in range(len(z_subset)))
#        q_tags.append(q_i)
#    
#    q_bar = vector(E, q_tags)
#    return Delta, q_bar, j

In [25]:
# Test the protocol
x_bar = vector(F, [1, 2])  # Prover's secret vector
print(f"Prover's secret vector x_bar: {x_bar}")

# Run Prover
t_vectors, messages, m_bar = prover(x_bar, Z)
print(f"Vetores t_i: {t_vectors}")
print(f"Mensagens para OT: {messages}")
print(f"Tags m_bar: {m_bar}")

# Run Verifier
Delta, q_bar, j = verifier(messages, Z)
print(f"Chave global Delta: {Delta}")
print(f"Tags q_bar: {q_bar}")
print(f"Índice j escolhido: {j}")

# Verify
verificar(x_bar, Delta, q_bar, m_bar)

Prover's secret vector x_bar: (1, 2)
Vetores t_i: [[1, 1, 4], [4, 0, 3]]
Mensagens para OT: [[1, 1, 4], [4, 0, 3]]
Tags m_bar: (0, 2*z + 3)
Protocolo LPN OT - 0
Verification successful.
Protocolo LPN OT - 1
Verification successful.
Chave global Delta: z + 1
Tags q_bar: (z + 1, 4*z)
Índice j escolhido: 2
Verificação bem-sucedida.


True

### Ex 2

In [26]:
from hashlib import shake_256

def simulate_xof(seed, num_elements, field):
    # Convert seed (vector in GF(2)) to bytes
    seed_bytes = bytes(int(seed[i]) for i in range(len(seed)))
    
    # Use SHAKE256 as XOF
    shake = shake_256()
    shake.update(seed_bytes)
    output_bytes = shake.digest(num_elements * 4)  # 4 bytes per element
    
    # Map bytes to elements of the field
    result = []
    field_order = field.order()
    for i in range(0, num_elements * 4, 4):
        # Extract 4 bytes and convert to integer
        chunk = output_bytes[i:i+4]
        int_val = int.from_bytes(chunk, byteorder='big') % field_order
        result.append(field(int_val))
        
    return vector(field, result[:num_elements])

### Protocolo ZK - sVOLE  em sistemas de equações polinomiais aleatórias

Neste modelo os polinómios $\,f_i(y\mathbin{;}x)\,$ são gerados de forma correlacionada com a chave privadas. Essencialmente a chave privada $\,w\,$ é escolhida em primeiro lugar; só depois são gerados os polinómios $\,f_i\,$ onde os termos de 1ª e 2ª ordem são gerados aleatoriamente e as constantes $\,c_i\,$ são calculadas de forma a que $\,w\,$ seja raíz desse polinómio.

`KeyGen`($\lambda$)

1. Sob input do parâmetro de segurança $\,\lambda\,$ são gerados os parâmetros
        - $\,p\,$ , $\,\kappa\,$ , característica do corpo primo $\,F\simeq \mathbb{F}_p\,$ e dimensão do corpo $\,E\,\simeq\,F^\kappa$
        - $\,n\,$ , número de variáveis de “input” , 
        -  $\,t\,$ , número de polinómios
        -  $\,N\,$ número  de elementos da base $\,\mathcal{Z}$   do protocolo SVOLE
        -  $\,M\,$ número de repetições
2. São geradas aleatóriamente duas “seeds”  $\,\rho,\mathsf{s} \in \{0,1\}^\lambda\,$  responsáveis por conjuntamente com um $\,\mathsf{XOF} \colon \,\{0,1\}^\lambda\times \mathbb{N}\,\to\,F^\ast\,$ gerar as chaves privadas e públicas.
3. A chave privada $\,w \in F^n\,$  é gerada como $\,w \gets \mathsf{XOF}(\rho,n)\,$
4. Com a “seed” $\,\mathsf{s}\,$ e parâmetro $\,N\,$ é gerado o conjunto $\,\mathcal{Z}\,\subset\,E\,$ , com $\,\#\mathcal{Z}\,=\,N\;$ , que vai ser usado no protocolo SVOLE.
5. Para cada $\,i\in [t]\,$  a constante $\,{c}_i\,$ e o triplo de vetores $\,(b_i,u_i,v_i)\;$ determinam o polinómio $\,f_i(y\mathbin{;}x)\,$. Assim
* Usando ainda a “seed” $\,\mathsf{s}\,$ gera-se um triplo  $\,(b_i,u_i,v_i) \in F^{3n}\,$  para cada $\,i\in [t]\,$.  Isto é
                        $$\,\quad\{b_i,u_i,v_i\}_{i\in [t]}\,\gets\, \mathsf{XOF}(\mathsf{s}\,,\,3\times n \times t)$$ 
* Calcula   $$\,\tilde{c_i} \gets  (b_i\ast w) + (u_i\ast w)\cdot (v_i\ast w)\,$$    para cada $\;i\in [t]$
6. A chave pública é o par $\,(\mathsf{s}\,,\,\mathsf{c})\quad\text{com}\;\mathsf{c} = \{-\tilde{c_i}\}_{j\in[t]}\;$. 


In [27]:
def KeyGen(lambda_, p, k, n, t, N):
    if not ZZ(p).is_prime():
        raise ValueError(f"p = {p} must be a prime number")
    if k <= 0:
        raise ValueError(f"k = {k} must be a positive integer")
    global E, F
    M = 10  # Número de repetições

    # 2. Gerar seeds rho e s em {0,1}^lambda
    rho = random_vector(GF(2), lambda_)
    s = random_vector(GF(2), lambda_)

    # 3. Gerar chave privada w \in F^n usando XOF(rho, n)
    w = simulate_xof(rho, n, F)

    # 4. Gerar conjunto Z \subset E com #Z = N usando XOF(s, N)
    Z_elements = simulate_xof(s, N, E)
    Z = list(set(Z_elements))
    while len(Z) < N:
        extra = E.random_element()
        if extra not in Z:
            Z.append(extra)
    Z = Z[:N]

    # 5. Gerar b_i, u_i, v_i e calcular c_i
    total_elements = 3 * n * t
    all_vectors = simulate_xof(s, total_elements, F)
    
    # Dividir all_vectors em t grupos de 3*n elementos
    b = []
    u = []
    v = []
    for i in range(t):
        start = i * 3 * n
        b_i = all_vectors[start:start + n]
        u_i = all_vectors[start + n:start + 2 * n]
        v_i = all_vectors[start + 2 * n:start + 3 * n]
        b.append(b_i)
        u.append(u_i)
        v.append(v_i)

    # 5.2. Calcular tilde_c_i
    tilde_c = [b[i].dot_product(w) + u[i].dot_product(w) * v[i].dot_product(w) for i in range(t)]

    # 6. Definir chave pública (s, c)
    c = [-tilde_c[i] for i in range(t)]
    public_key = (s, c)

    return w, Z, b, u, v, public_key, M


`Commit`
1. O Verifier  gera aleatoriamente  a chave global $\,\Delta\,\gets\,\mathcal{Z}$ usada nesta sessão.
2. O Prover e o Verifier  executam o protocolo $\,\mathsf{sVOLE}\,$ para a chave privada
                                     $$\,w = \{w_k\}_{k\in[n]}\,\in\,F^n\,$$  
    Como resultado 
        1. o Prover,  passa a conhecer a “tag” vetorial $$\,\tau = \{\tau_k\}_{k\in[n]}\,\in\, E^n\,\simeq\,F^{\kappa\times n}$$. 
        2. O Verifier  passa a conhecer o vetor de  “tags”  $$\,\omega\,=\,\{\omega_k\}_{k\in[n]}\,\in\,E^n\simeq F^{\kappa\times n}$$    tais que 
                                $$\omega_k\,=\,w_k\cdot\Delta + \tau_k\quad$$ 
                                para todo $\,k\in[n]$
3. O Prover  gera aleatoriamente máscaras $\,\mu = \{\mu_i\}_{i\in[t]}\gets F^t\,$  e, conjuntamente, com o Verifier, entra num protocolo $\,\mathsf{sVOLE}\,$ para cada $\,\mu_i\,$ e para a chave global $\,\Delta\,$.  Daqui
    1. o Prover recebe  “tags” $\,\{\zeta_i\}_{i\in[t]}\,$ 
    2. o Verifier recebe “tags”  $\,\{\eta_i\}_{i\in[t]}\,$  tais que $\eta_i \,=\,\mu_i\cdot \Delta + \zeta_i\qquad$ com $\;\eta_i\,,\,\zeta_i\,\in\,E$ 
4. Para cada polinómio $\,\{f_i\,\}_{i\in[t]}$, o Prover computa 
  $$\quad A_{1,i} \,\gets\,b_i\ast \tau + (u_i\ast w)\cdot(v_i\ast \tau) + (v_i\ast w)\cdot (u_i\ast \tau)\;$$
  
  $$\;A_{0,i}\,\gets\; (u_i\ast\tau)\cdot(v_i\ast\tau)\,$$
5.  Para cada polinómio $\,\{f_i\,\}_{i\in[t]}$,  o Verifier calcula   $\;B_{i}\,\gets\,f_i(\Delta, \omega)\, = A_{1,i} \cdot \Delta + A_{0,i}$

In [28]:
def Commit(w, Z, b, u, v, c, p, k, t):
    global E, F

    # 1. Verifier escolhe Delta
    Delta = Z[randrange(len(Z))]
    while Delta == 0:  # Safety check
        Delta = Z[randrange(len(Z))]

    # 2. Executar sVOLE para w
    _, messages_w, tau = prover(w, Z)
    _, omega, _ = verifier(messages_w, Z, Delta)

    # 3. Gerar mu e executar sVOLE para cada mu_i
    mu = random_vector(F, t)
    zeta = []
    eta = []
    for i in range(t):
        mu_i_vec = vector(F, [mu[i]])  # mu_i como vetor de dimensão 1
        _, msgs, zeta_i = prover(mu_i_vec, Z)
        _, eta_i, _ = verifier(msgs, Z, Delta)
        zeta.append(zeta_i[0])  
        eta.append(eta_i[0])    

    # 4. Prover calcula A_{1,i} e A_{0,i}$$
    A1 = []
    A0 = []

    for i in range(t):
        A1_i = (b[i] * tau) + (u[i] * w) * (v[i] * tau) + (u[i] * tau) * (v[i] * w) 
        A0_i = (u[i] * tau) * (v[i] * tau)  
        A1.append(A1_i)
        A0.append(A0_i)

    # 5. Verifier computes B_i = A_{1,i} * Delta + A_{0,i}
    B = []
    for i in range(t):
        B_i = A1[i] * Delta + A0[i]
        B.append(B_i)
    return Delta, tau, omega, mu, zeta, eta, A1, A0, B

Em termos de implementação note-se que
Cada $\,w_k\,$ é um elemento de $\,F\,$ mas, tanto $\,\Delta\,$ como cada $\,\omega_k\,$ e $\,\tau_k\,$,  identifica-se com  um vetor de $\,\kappa\,$ componentes de elementos de $\,F\,$
Igualmente cada $\,\mu_i\,$ é um elemento de $\,F\,$ mas cada $\,\eta_i\,,\,\zeta_i\,,\,A_{1,i}\,,\,A_{0,i}\;$ identifica-se com um vetor de $\,\kappa\,$ componentes de elementos de $\,F\,$.
Em termos vetoriais $\,w\,$ é descrito por um vetor em  $\,F^n\,$ enquanto que $\,\Delta\,,\,\omega\,,\,\tau\;$ são descritos por matrizes $\,F^{\kappa\times n}\,$.  Igualmente $\,\mu\,$ é descrito por um vetor em $\,F^t\,$ enquanto que $\,\eta\,,\,\zeta\;$ são descritos por  matrizes $\,F^{\kappa\times t}$.

`Challenge`


O Verifier torna pública uma “seed” aleatoriamente gerada $\,\mathsf{e}\,\gets\,\{0,1\}^\lambda$. Com esta “seed” e um XOF  o Verifier  gera um vetor $\,\chi \in E^t\,$ e calcula sucessivamente
* $\,\mathsf{B}\,\gets\,\sum_{j\in[t]}\,\chi_j\cdot B_j\,$
* $\,\eta^\ast \,\gets\, \sum_{j\in[t]}\,\chi_j\cdot\eta_j$
* $\,\mathsf{B}^\ast \,\gets\, \mathsf{B}\,+\,\eta^\ast$

In [29]:
def Challenge(t, E, lambda_, B, eta):
    # Gerar seed e
    e = random_vector(GF(2), lambda_)
    
    # Gerar chi usando XOF simulado
    chi = simulate_xof(e, t, E)
    
    # Calcular B, eta^* e B^*
    B_sum = sum(chi[j] * B[j] for j in range(t))
    eta_star = sum(chi[j] * eta[j] for j in range(t))
    B_star = B_sum + eta_star
    
    return e, chi, B_star

`Proove`


1. O Prover recebe o desafio $\,\mathsf{e}\,$ e com essa “seed”  e o mesmo XOF gera o mesmo vetor $\,$ $\,\chi\,$
2. Calcula sucessivamente
* $\,\mathsf{}$ $\,\mathsf{A}_1 \,\gets\, \sum_{j\in[t]}\,\chi_j\cdot  A_{1,j}\quad$  e   $\quad\;\mathsf{A}_0 \,\gets\, \sum_{j\in[t]}\,\chi_j\cdot  A_{0,j}\;$

* $\;\mu^\ast \;\;\gets\,\sum_{j\in[t]}\,\chi_j\cdot \mu_j\qquad$ e $\qquad \zeta^\ast \;\;\gets\,\sum_{j\in[t]}\,\chi_j\cdot \zeta_j$

* $\;\mathsf{A}^\ast_1 \,\gets\,\mathsf{A}_1\,+\,\mu^\ast\qquad\qquad\;\;$ e $\qquad \mathsf{A}^\ast_0\,\gets\,\mathsf{A}_0\,+\,\zeta^\ast$

e envia os valores $\;\mathsf{A}_1^\ast\;$ e $\;\mathsf{A}_0^\ast\;$  ao Verifier.

In [30]:
def Prove(e, chi, A1, A0, mu, zeta, t, E):
    A1_sum = sum(E(chi[j]) * A1[j] for j in range(t))
    A0_sum = sum(E(chi[j]) * A0[j] for j in range(t))
    mu_star = sum(chi[j] * E(mu[j]) for j in range(t))
    zeta_star = sum(chi[j] * zeta[j] for j in range(t))
    
    A1_star = A1_sum + mu_star
    A0_star = A0_sum + zeta_star
    
    return A1_star, A0_star

`Verify`

O Verifier verifica  $\;\mathsf{B}^\ast \mathbin{\,\stackrel{?}=\,} \mathsf{A}_1^\ast\cdot\Delta + \mathsf{A}_0^\ast$

In [31]:
def Verify(B_star, Delta, A1_star, A0_star):
    return B_star == A1_star * Delta + A0_star

### Correr Código

In [32]:
# Definir parâmetros
p = 5
k = 2
N = 3
lambda_ = 16
t = 2
n = 2

# Gerar chaves
w, Z, b, u, v, (s, c), M = KeyGen(lambda_, p, k, n, t, N)
print(f"Chave privada w: {w}")
print(f"Subconjunto Z: {Z}")

# Commit
Delta, tau, omega, mu, zeta, eta, A1, A0, B = Commit(w, Z, b, u, v, c, p, k, t)
print(f"Delta: {Delta}")

# Challenge
e, chi, B_star = Challenge(t, E, lambda_, B, eta)
print(f"Seed e: {e[:5]}...")
print(f"Chi: {chi}")
print(f"B_star: {B_star}")

# Prove
A1_star, A0_star = Prove(e, chi, A1, A0, mu, zeta, t, E)
print(f"A1_star: {A1_star}")
print(f"A0_star: {A0_star}")

# Verify
result = Verify(B_star, Delta, A1_star, A0_star)
print(f"Verificação: {result}")

Chave privada w: (3, 1)
Subconjunto Z: [0, 4, 2*z + 1]
Protocolo LPN OT - 0
Verification successful.
Protocolo LPN OT - 1
Verification successful.
Protocolo LPN OT - 0
Verification successful.
Protocolo LPN OT - 0
Verification successful.
Delta: 2*z + 1
Seed e: (1, 1, 1, 0, 1)...
Chi: (0, 4)
B_star: 4
A1_star: 4*z + 2
A0_star: 4*z + 3
Verificação: True
